In [1]:
import pygplates
import sys
import glob, re
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import xarray as xr
import points_spatial_tree

import polygon_processing as pp

import paleogeography as pg
reload(pg)

%matplotlib inline



basedir = '/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles/'

sampling = 1.0

rotation_model = pygplates.RotationModel(['%s/Global_EB_250-0Ma_GK07_Matthews++.rot' % basedir,
                                          '%s/Global_EB_410-250Ma_GK07_Matthews++.rot' % basedir])


# make a sorted list of the (midpoint) times for paleogeography polygons
tmp = glob.glob('%s/*/' % basedir)

time_list = []
for tm in tmp:
    time_list.append(float(re.findall(r'\d+Ma+',tm)[1][:-2]))

time_list.sort()



In [31]:
def get_masked_multipoint(coords,masking_array,plate_partitioner,valid_time=None):
    
    multipoint_feature = pygplates.Feature()
    multipoint_feature.set_geometry(pygplates.MultiPointOnSphere(zip(np.array(coords[0])[masking_array],
                                                                     np.array(coords[1])[masking_array])))  
    (pg_points_masked,
     dummy) = plate_partitioner.partition_features(multipoint_feature,
                                                   partition_return = pygplates.PartitionReturn.separate_partitioned_and_unpartitioned)
        
    if valid_time is not None:
        for feature in pg_points_masked:
            feature.set_valid_time(valid_time[0],valid_time[1])
        
    return pg_points_masked


def get_change_masks(t1,points,spatial_tree_of_uniform_recon_points,psl_t1,psl_t2,rotation_model):
                    
    distance_to_land_t1,distance_to_psl_t1 = pp.run_grid_pnp(t1,
                                                             points,
                                                             spatial_tree_of_uniform_recon_points,
                                                             psl_t1,
                                                             rotation_model)
    
    distance_to_land_t2,distance_to_psl_t2 = pp.run_grid_pnp(t1,
                                                             points,
                                                             spatial_tree_of_uniform_recon_points,
                                                             psl_t2, 
                                                             rotation_model)
    
    # this mask is true where one (and only one) of the indicators is not zero
    # --> delineates points that change environment between time steps
    #msk = np.logical_xor(distance_to_land_t1,distance_to_land_t2)
    
    regression_msk = np.logical_and(distance_to_land_t1==0,distance_to_land_t2>0)
    
    transgression_msk = np.logical_and(distance_to_land_t1>0,distance_to_land_t2==0)
    
    always_land_msk = np.logical_and(distance_to_land_t1==0,distance_to_land_t2==0)
    
    
    return distance_to_land_t1,distance_to_psl_t1,distance_to_land_t2,distance_to_psl_t2,\
           regression_msk,transgression_msk,always_land_msk


# function to run point in/near polygon test for two successive time slices
def get_change_mask_multipoints(pg_features,t1,t2,psl_t1,psl_t2,
                                points,spatial_tree_of_uniform_recon_points,
                                rotation_model,plot=False):
    
    print 'Working on interpolation from %0.2f Ma to %0.2f Ma .....' % (t1,t2)
        
    plate_partitioner = pygplates.PlatePartitioner(pg_features, rotation_model, reconstruction_time=t1)
    
    (distance_to_land_t1,
     distance_to_psl_t1,
     distance_to_land_t2,
     distance_to_psl_t2,
     regression_msk,
     transgression_msk,
     always_land_msk) = get_change_masks(t1,points,spatial_tree_of_uniform_recon_points,
                                         psl_t1,psl_t2,rotation_model)
    
    coords = zip(*[point.to_lat_lon() for point in points])
    
    pg_points_regression = get_masked_multipoint(coords,regression_msk,plate_partitioner,
                                                 valid_time=[t2,t1+0.01])
    pg_points_transgression = get_masked_multipoint(coords,transgression_msk,plate_partitioner,
                                                    valid_time=[t2,t1+0.01])
    pg_points_always_land = get_masked_multipoint(coords,always_land_msk,plate_partitioner,
                                                  valid_time=[t2,t1])

    pygplates.FeatureCollection(pg_points_regression).write('./tween_feature_collections/mountain_regression_%0.2fMa_%0.2fMa.gpmlz' % (t1,t2))
    pygplates.FeatureCollection(pg_points_transgression).write('./tween_feature_collections/mountain_transgression_%0.2fMa_%0.2fMa.gpmlz' % (t1,t2))
    pygplates.FeatureCollection(pg_points_always_land).write('./tween_feature_collections/mountain_stable_%0.2fMa_%0.2fMa.gpmlz' % (t1,t2))


    

# function to run point in/near polygon test for two successive time slices
def interpolate_paleoshoreline_for_stage(pg_features,t1,t2,psl_t1,psl_t2,time_step,
                                         points,spatial_tree_of_uniform_recon_points,
                                         rotation_model,plot=False):
    
    print 'Working on interpolation from %0.2f Ma to %0.2f Ma .....' % (t1,t2)
    
    plate_partitioner = pygplates.PlatePartitioner(pg_features, rotation_model, reconstruction_time=t1)
    
    (distance_to_land_t1,
     distance_to_psl_t1,
     distance_to_land_t2,
     distance_to_psl_t2,
     regression_msk,
     transgression_msk,
     always_land_msk) = get_change_masks(t1,points,spatial_tree_of_uniform_recon_points,
                                         psl_t1,psl_t2,rotation_model)
    

    # normalised distance derivation
    # for each point, divide the distance to shoreline at t0 by the total distance to both shorelines
    # --> if the point is halfway between the shorelines, value will be 0.5
    #     if the point is closer to the t1 shoreline, the value will be less than 0.5
    #     all values will be between 0 and 1
    psl_dist_norm = np.divide(distance_to_psl_t1,(distance_to_psl_t1+distance_to_psl_t2))

    
    t_diff = (t2-t1)
    
    
    if plot:
        coords = zip(*[point.to_lat_lon() for point in points])
        plt.figure(figsize=(20,9))
        plt.scatter(np.array(coords[1]),np.array(coords[0]),c=distance_to_land_t1,
                    edgecolors='',s=2,alpha=0.5,vmin=0.,vmax=0.1)
        plt.title('%0.2f - %0.2f Ma' % (t1,t2))
        plt.colorbar()
        plt.show()
    
        coords = zip(*[point.to_lat_lon() for point in points])
        plt.figure(figsize=(20,9))
        plt.scatter(np.array(coords[1]),np.array(coords[0]),c=distance_to_psl_t1,
                    edgecolors='',s=2,alpha=0.5,vmin=0.,vmax=0.1)
        plt.title('%0.2f - %0.2f Ma' % (t1,t2))
        plt.colorbar()
        plt.show()

        coords = zip(*[point.to_lat_lon() for point in points])
        plt.figure(figsize=(20,9))
        plt.scatter(np.array(coords[1]),np.array(coords[0]),c=distance_to_land_t2,
                    edgecolors='',s=2,alpha=0.5,vmin=0.,vmax=0.1)
        plt.title('%0.2f - %0.2f Ma' % (t1,t2))
        plt.colorbar()
        plt.show()

        coords = zip(*[point.to_lat_lon() for point in points])
        plt.figure(figsize=(20,9))
        plt.scatter(np.array(coords[1]),np.array(coords[0]),c=distance_to_psl_t2,
                    edgecolors='',s=2,alpha=0.5,vmin=0.,vmax=0.1)
        plt.title('%0.2f - %0.2f Ma' % (t1,t2))
        plt.colorbar()
        plt.show()
    
    
    # before looping over the time steps, create an empty list to put the features in
    pg_points_land_list = []
    pg_points_marine_list = []
    
    # don't need to do t2 itself, since this will be first step in next iteration
    for reconstruction_time in np.arange(t1,t2,time_step):
        
        if reconstruction_time==t1:
            land_points = np.where(distance_to_land_t1==0)[0]
            
        else:
            # normalised time, in range 0 to 1 between start and end of stage
            t_norm = (reconstruction_time-t1)/t_diff   
            
            is_transgressing_land_msk = np.less_equal(psl_dist_norm,t_norm)
            
            is_regressing_land_msk = np.greater_equal(psl_dist_norm,t_norm)
            
            land_points = np.where(
                np.logical_or(
                    np.logical_or(
                        np.logical_and(is_regressing_land_msk,regression_msk),
                        np.logical_and(is_transgressing_land_msk,transgression_msk)
                    ),
                always_land_msk))[0]
            
            
            #print t_norm


        marine_mask = np.ones(distance_to_land_t1.shape,dtype=bool)
        marine_mask[land_points] = False
        marine_points = np.where(marine_mask)
            
        coords = zip(*[point.to_lat_lon() for point in points])


        if plot:
            plt.figure(figsize=(20,9))
            plt.plot(np.array(coords[1])[land_points],np.array(coords[0])[land_points],'k.',markersize=3,alpha=0.3)
            plt.plot(np.array(coords[1])[marine_points],np.array(coords[0])[marine_points],'r.',markersize=3,alpha=0.3)
            plt.title('%0.2fMa' % reconstruction_time)
            plt.xlim(-180.,180.)
            plt.ylim(-90.,90.)
            plt.savefig('./tmp/interpolated_psl_%0.2fMa.png' % reconstruction_time)
            plt.close()
            

        #multipoint_feature = pygplates.Feature()
        #multipoint_feature.set_geometry(pygplates.MultiPointOnSphere(zip(np.array(coords[0])[land_points],
        #                                                                 np.array(coords[1])[land_points])))
        #
        #(pg_points_land,
        # dummy) = plate_partitioner.partition_features(multipoint_feature,
        #                                               partition_return = pygplates.PartitionReturn.separate_partitioned_and_unpartitioned)
        #
        #multipoint_feature = pygplates.Feature()
        #multipoint_feature.set_geometry(pygplates.MultiPointOnSphere(zip(np.array(coords[0])[marine_points],
        #                                                                 np.array(coords[1])[marine_points])))
        #
        #(pg_points_marine,
        # dummy) = plate_partitioner.partition_features(multipoint_feature,
        #                                               partition_return = pygplates.PartitionReturn.separate_partitioned_and_unpartitioned)
        #
        #for feature in pg_points_land:
        #    feature.set_valid_time(reconstruction_time+(time_step/2.),
        #                           reconstruction_time-(time_step/2.)+0.01)
        #for feature in pg_points_marine:
        #    feature.set_valid_time(reconstruction_time+(time_step/2.),
        #                           reconstruction_time-(time_step/2.)+0.01)
            
        pg_points_land = get_masked_multipoint(coords,land_points,plate_partitioner,
                                               valid_time=[reconstruction_time+(time_step/2.),
                                               reconstruction_time-(time_step/2.)+0.01])
        
        pg_points_marine = get_masked_multipoint(coords,marine_points,plate_partitioner,
                                                 valid_time=[reconstruction_time+(time_step/2.),
                                                 reconstruction_time-(time_step/2.)+0.01])
        
        
        # append the point features for this time to the overall list
        pg_points_land_list+=pg_points_land
        pg_points_marine_list+=pg_points_marine
        
    
    pygplates.FeatureCollection(pg_points_land_list).write('./tween_feature_collections/tweentest_mountain_%0.2fMa_%0.2fMa.gpmlz' % (t1,t2))
    #pygplates.FeatureCollection(pg_points_marine_list).write('./tween_feature_collections/tweentest_ocean_%0.2fMa_%0.2fMa.gpmlz' % (t1,t2))

    


In [32]:
area_threshold = 0.001
time_step = 1


for t1,t2 in zip(time_list[:-1],time_list[1:]):
    
    #env_list = ['lm','m','i']
    env_list = ['m']

    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,t1)

    pg_features = pg.load_paleogeography(pg_dir,env_list)
    cf = pp.merge_polygons(pg_features,rotation_model,time=t1,sampling=0.25)
    sieve_polygons_t1 = pp.polygon_area_threshold(cf,area_threshold)

    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,t2)

    pg_features = pg.load_paleogeography(pg_dir,env_list)
    #cf2 = []
    for f in pg_features:
        f.set_valid_time(9999,-9999)
        #cf2.append(f)
    cf = pp.merge_polygons(pg_features,rotation_model,time=t1,sampling=0.25)

    sieve_polygons_t2 = pp.polygon_area_threshold(cf,area_threshold)



    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,t1)
    env_list = ['lm','m','i','sm']
    pg_features = pg.load_paleogeography(pg_dir,env_list)


    lons,lats = np.meshgrid(np.arange(-180,180+sampling,sampling),np.arange(-90,90+sampling,sampling))
    points = [pygplates.PointOnSphere(lat, lon) for lat, lon in zip(lats.flatten(),lons.flatten())]

    spatial_tree_of_uniform_recon_points = points_spatial_tree.PointsSpatialTree(points)

    #interpolate_paleoshoreline_for_stage(pg_features,t1,t2,sieve_polygons_t1,sieve_polygons_t2,time_step,points,
    #                                     spatial_tree_of_uniform_recon_points,rotation_model,plot=False)
    
    get_change_mask_multipoints(pg_features,t1,t2,sieve_polygons_t1,sieve_polygons_t2,
                                points,spatial_tree_of_uniform_recon_points,
                                rotation_model,plot=False)



['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_14Ma/m_fig62_20_11_PresentDay_Paleogeog_Matthews2016_14.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/lm_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-2Ma_Shapefiles//PresentDay_Paleogeog_Matthews2016_6Ma/m_fig64_11_2_PresentDay_Paleogeog_Matthews2016_6.00Ma.shp']
['/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/Paleogeography_Matthews2016_410-